In [1]:
from hamiltonian import HamiltonianSmall, Hamiltonian

In [2]:
lih = HamiltonianSmall('LiH', 1.5)
beh2 = HamiltonianSmall('BeH2', 1.3)

h2_jw = Hamiltonian('H2_6-31G_8qubits', 'jw')
h2_parity = Hamiltonian('H2_6-31G_8qubits', 'parity')
h2_bk = Hamiltonian('H2_6-31G_8qubits', 'bk')

water_jw = Hamiltonian('H2O_STO3g_14qubits', 'jw')
water_parity = Hamiltonian('H2O_STO3g_14qubits', 'parity')
water_bk = Hamiltonian('H2O_STO3g_14qubits', 'bk')

ammonia_jw = Hamiltonian('NH3_STO3g_16qubits', 'jw')
ammonia_parity = Hamiltonian('NH3_STO3g_16qubits', 'parity')
ammonia_bk = Hamiltonian('NH3_STO3g_16qubits', 'bk')

In [3]:
%time lih.pauli_rep.ground()
%time beh2.pauli_rep.ground()
%time h2_jw.pauli_rep.ground()
%time energy, state = water_jw.pauli_rep.ground()
# %time energy, state = ammonia_jw.pauli_rep.ground() # takes about 4 minutes

CPU times: user 112 ms, sys: 2.21 ms, total: 115 ms
Wall time: 113 ms
CPU times: user 261 ms, sys: 2.98 ms, total: 264 ms
Wall time: 261 ms
CPU times: user 540 ms, sys: 32.2 ms, total: 572 ms
Wall time: 413 ms
CPU times: user 23.3 s, sys: 3.13 s, total: 26.4 s
Wall time: 10.1 s


In [4]:
%time lih.pauli_rep.ground(multithread=True)
%time beh2.pauli_rep.ground(multithread=True)
%time h2_jw.pauli_rep.ground(multithread=True)
%time energy, state = water_jw.pauli_rep.ground(multithread=True)
# %time energy, state = ammonia_jw.pauli_rep.ground(multithread=True) # takes about 1 minute

CPU times: user 34 ms, sys: 50.5 ms, total: 84.5 ms
Wall time: 408 ms
CPU times: user 34.4 ms, sys: 42.8 ms, total: 77.2 ms
Wall time: 421 ms
CPU times: user 182 ms, sys: 76.5 ms, total: 258 ms
Wall time: 470 ms
CPU times: user 14.9 s, sys: 1.88 s, total: 16.8 s
Wall time: 3.58 s


In [5]:
from sparse import energy as sparseenergy
from sparse import energy_kron

pr = beh2.pauli_rep
energy, state = pr.ground()
print("energy: ", energy)
%time energy_sparse = sparseenergy(pr, state)
print("energy_sparse: ", energy_sparse)
#%time energy_kron = energy_kron(pr, state)
#print("energy_kron: ", energy_kron)

energy:  0.8494938395630901
CPU times: user 260 ms, sys: 2.88 ms, total: 263 ms
Wall time: 261 ms
energy_sparse:  0.8494938395630887


In [6]:
β = pr.local_dists_uniform()
%time pr.variance_local(energy, state, β)

CPU times: user 361 ms, sys: 1.69 ms, total: 362 ms
Wall time: 361 ms


22.470145625464756

In [7]:
print("ell_1: ", pr.variance_ell_1(energy))

β = pr.local_dists_pnorm(1)
print("1_norm: ", pr.variance_local(energy, state, β))

β = pr.local_dists_pnorm(2)
print("2_norm: ", pr.variance_local(energy, state, β))

β = pr.local_dists_pnorm('infinity')
print("max_norm: ", pr.variance_local(energy, state, β))

ell_1:  44.84428759596599
1_norm:  92.75535492849629
2_norm:  84.12338038780389
max_norm:  82.42512285439781


# Variance optimisation (old and wrong, but convex)

If we use a uniform/biased pauli/magic bases algorithm to estimate the energy, and we assume that the state is a product-state (or close to a produce-state) then we end up wanting to minimise:
$$
    \Var[\nu]
    =
    \sum_{\Qarrow} \alpha_\Qarrow^2 \prod_{i\in\supp(\Qarrow)} \beta_{i,Q_i}^{-1}
    +
    \sum_{\Qarrow,\Rarrow \in \{I,Z\}^{\otimes n}} \alpha_\Qarrow \alpha_\Rarrow \prod_{i} m_i(1-\delta_{Q_i R_i})
    - \tr(H_0\rho)^2
$$
(Above is the chemistry HF situation, where the product-state is in the $Z$-basis, but this statement wouldn't depend on using $Z$-basis uniformly.)

We end up wanting to minimise:
$$
    \sum_{\Qarrow} \alpha_\Qarrow^2 \prod_{i\in\supp(\Qarrow)} \beta_{i,Q_i}^{-1}
    \qquad
    \textrm{subject to}
    \qquad
    \beta_{i,X}+\beta_{i,Y}+\beta_{i,Z}=1 \,\forall i,
    \qquad
    \beta_{i,P}\ge 0
$$

In [8]:
β_optimal = pr.local_dists_optimal()
print(pr.variance_local(energy, state, β_optimal))

/opt/anaconda3/envs/qiskit18/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


16.194135745577128


In [9]:
def variances_dict(pauli_rep):
    pr = pauli_rep
    dic = {}
    
    energy, state = pr.ground(multithread=True)
    print("energy :", energy)

    # ell_1
    var = pr.variance_ell_1(energy)
    print("ell 1: ", var)
    dic['ell_1'] = var
    
    # uniform
    β = pr.local_dists_uniform()
    var = pr.variance_local(energy, state, β)
    print("uniform: ", var)
    dic['uniform'] = var
    
    # optimal
    β = pr.local_dists_optimal()
    var = pr.variance_local(energy, state, β)
    print("optimal: ", var)
    dic['optimal'] = var
    
    return dic

In [10]:
variances_ALL = {}
beta_optimal_ALL = {}

In [11]:
name = 'lih'
pr = lih.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -0.6791966848293151
ell 1:  8.826402522940885
uniform:  2.24717620961014
optimal:  1.5743727896616164
CPU times: user 1.11 s, sys: 191 ms, total: 1.3 s
Wall time: 672 ms


In [12]:
name = 'beh2'
pr = beh2.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : 0.8494938395630852
ell 1:  44.84428759596597
uniform:  22.4701456254646
optimal:  16.194135745577018
CPU times: user 4.62 s, sys: 554 ms, total: 5.17 s
Wall time: 2.06 s


In [13]:
name = 'h2_jw'
pr = h2_jw.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -1.8608605555207673
ell 1:  119.67906001905904
uniform:  51.39982021387603
optimal:  17.742071936811385
CPU times: user 9.39 s, sys: 1.02 s, total: 10.4 s
Wall time: 3.21 s


In [14]:
name = 'h2_parity'
pr = h2_parity.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -1.860860555520758
ell 1:  119.67906001905918
uniform:  70.7614744196227
optimal:  18.924750112035056
CPU times: user 34.3 s, sys: 4.18 s, total: 38.5 s
Wall time: 8.93 s


In [15]:
name = 'h2_bk'
pr = h2_bk.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -1.8608605555207645
ell 1:  119.67906001905939
uniform:  168.86600324405487
optimal:  19.51693242093154
CPU times: user 20.2 s, sys: 2.73 s, total: 23 s
Wall time: 5.8 s


In [16]:
pr = water_jw.pauli_rep
%time β = pr.local_dists_optimal()
β

CPU times: user 6min 45s, sys: 1min 8s, total: 7min 53s
Wall time: 1min


{0: [0.43257858556565915, 0.43257858722135467, 0.1348428272129862],
 1: [0.24098007537433425, 0.2409800708577077, 0.518039853767958],
 2: [0.13870541903721406, 0.13870542588015417, 0.7225891550826317],
 3: [0.13957872152763764, 0.13957871134125624, 0.7208425671311062],
 4: [0.16984472160573807, 0.1698447243114196, 0.6603105540828423],
 5: [0.2675208449775755, 0.2675208434379076, 0.464958311584517],
 6: [0.11647067744295292, 0.11647067819430176, 0.7670586443627454],
 7: [0.43257857708372555, 0.43257860312684154, 0.13484281978943294],
 8: [0.24098005034367995, 0.24098008250406633, 0.5180398671522537],
 9: [0.13870542248842968, 0.13870542724539398, 0.7225891502661764],
 10: [0.13957872657623138, 0.13957872195005588, 0.7208425514737127],
 11: [0.16984475333796817, 0.16984473040786618, 0.6603105162541657],
 12: [0.2675208758292949, 0.26752084565650264, 0.46495827851420257],
 13: [0.11647066971784335, 0.1164706702788042, 0.7670586600033525]}

In [17]:
%time energy, state = pr.ground(multithread=True)

CPU times: user 23.7 s, sys: 3.55 s, total: 27.2 s
Wall time: 4.82 s


In [18]:
%time var = pr.variance_ell_1(energy)
print("ell 1: ", var)

CPU times: user 15 µs, sys: 3 µs, total: 18 µs
Wall time: 22.2 µs
ell 1:  4363.497773126067


In [19]:
%time var = pr.variance_local(energy, state, pr.local_dists_uniform(), multithread=True)
print("uniform: ", var)

CPU times: user 33.3 ms, sys: 55.7 ms, total: 89 ms
Wall time: 49 s
uniform:  2839.0394682189617


In [20]:
%time var = pr.variance_local(energy, state, β, multithread=True)
print("optimal: ", var)

CPU times: user 36.2 ms, sys: 51.5 ms, total: 87.7 ms
Wall time: 49.3 s
optimal:  257.5450993727777
